<a href="https://colab.research.google.com/github/sergio-g-rubio/baseball_stats/blob/main/Previsao_Logistica_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚚 Previsão de Tempo de Operação Logística com Machine Learning
Este notebook treina um modelo de regressão para prever a duração de operações logísticas com base em atributos como tipo de operação, produto, peso, turno, etc.

In [66]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# 📂 Carregamento e Visualização do Dataset Logístico

In [62]:
# Carregar dataset (faça upload manual do CSV gerado anteriormente)
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(next(iter(uploaded)))
df.head()

Saving dataset_logistica_1000_linhas.csv to dataset_logistica_1000_linhas (3).csv


,tipo_operacao,tipo_produto,quantidade,peso_total_kg,turno,hora_inicio,dia_semana,tempo_total_min
0,recebimento,paletizado,228,587.37,tarde,12:00,quinta,42
1,expedição,paletizado,156,145.25,tarde,16:30,sábado,41
2,recebimento,fracionado,229,657.76,tarde,12:00,quarta,41
3,recebimento,matéria-prima,239,550.00,manhã,6:30,quinta,37
4,expedição,fracionado,125,321.97,tarde,16:45,terça,46


In [63]:
# Separar variáveis independentes (X) e alvo (y)
X = df.drop(["tempo_total_min", "hora_inicio"], axis=1)  # Remove a string 'hora_inicio'
y = df["tempo_total_min"]

In [64]:
# Definir colunas categóricas e numéricas
cat_cols = ["tipo_operacao", "tipo_produto", "turno", "dia_semana"]
num_cols = ["quantidade", "peso_total_kg", "hora_minutos"]

# Pré-processamento
preprocessador = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown='ignore'), cat_cols)
], remainder='passthrough')

In [65]:
# Pipeline com Random Forest
modelo = Pipeline([
    ("pre", preprocessador),
    ("rf", RandomForestRegressor(n_estimators=100, random_state=42))
])

# Dividir em treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar
modelo.fit(X_train, y_train)

# Prever
y_pred = modelo.predict(X_test)

# Avaliação
mae = mean_absolute_error(y_test, y_pred)
print(f"O modelo erra, em média, cerca de {mae:.2f} minutos ao prever a duração de uma operação logística.")
print("Esse valor representa a diferença média entre o tempo real e o tempo previsto, em minutos.")

O modelo erra, em média, cerca de 5.50 minutos ao prever a duração de uma operação logística.
Esse valor representa a diferença média entre o tempo real e o tempo previsto, em minutos.


# 📊 Avaliação de Desempenho do Modelo Preditivo Logístico

In [59]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Cálculos das métricas
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
erro_percentual_medio = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("📊 Resultado da Análise do Modelo de Previsão Logística:\n")

print(f"➡️ Em média, o modelo erra {mae:.2f} minutos na estimativa do tempo de operação.")
print(f"➡️ O desvio médio dos erros mais graves (RMSE) é de {rmse:.2f} minutos.")
print(f"➡️ Em termos proporcionais, o erro médio representa cerca de {erro_percentual_medio:.2f}% do tempo real.")
print(f"➡️ O modelo consegue explicar aproximadamente {r2:.2%} da variação dos tempos reais registrados.")

print("\n🔍 Como interpretar esses números:")

if erro_percentual_medio < 5:
    print("✅ As previsões do modelo são muito próximas da realidade. Isso é excelente para planejamento e monitoramento de SLAs.")
elif erro_percentual_medio < 10:
    print("🟡 As previsões são boas, com alguns desvios aceitáveis. Pode ser útil para gerar estimativas em painéis logísticos.")
else:
    print("🔴 O modelo ainda apresenta certa imprecisão. Isso pode ser causado por variáveis importantes que estão faltando, dados fora do padrão ou por inconsistência nas operações.")

print("\n💡 Sugestões para melhorar:")
print("- Verifique se há informações importantes que não estão sendo usadas (ex: tipo de cliente, SKU, equipamento).")
print("- Remova valores muito fora do padrão (outliers) que distorcem a média.")
print("- Considere separar modelos por tipo de operação ou turno, se os comportamentos forem muito diferentes.")


📊 Resultado da Análise do Modelo de Previsão Logística:

➡️ Em média, o modelo erra 5.50 minutos na estimativa do tempo de operação.
➡️ O desvio médio dos erros mais graves (RMSE) é de 6.47 minutos.
➡️ Em termos proporcionais, o erro médio representa cerca de 8.92% do tempo real.
➡️ O modelo consegue explicar aproximadamente 93.20% da variação dos tempos reais registrados.

🔍 Como interpretar esses números:
🟡 As previsões são boas, com alguns desvios aceitáveis. Pode ser útil para gerar estimativas em painéis logísticos.

💡 Sugestões para melhorar:
- Verifique se há informações importantes que não estão sendo usadas (ex: tipo de cliente, SKU, equipamento).
- Remova valores muito fora do padrão (outliers) que distorcem a média.
- Considere separar modelos por tipo de operação ou turno, se os comportamentos forem muito diferentes.


# 🧪 Simulação de Dimensionamento de Equipe com Base em Previsão de Tempo Operacional

In [70]:
import numpy as np
import pandas as pd

fator_escala = (operacoes_turno["quantidade"] / 1000).clip(lower=1)
operacoes_turno["tempo_previsto_corrigido"] = operacoes_turno["tempo_previsto_min"] * fator_escala

# ⚙️ Simulação: lista de operações em um turno
operacoes_turno = pd.DataFrame([
    {'tipo_operacao': 'recebimento', 'tipo_produto': 'matéria-prima', 'quantidade': 15000, 'peso_total_kg': 8000, 'turno': 'manhã', 'dia_semana': 'segunda', 'hora_inicio': '08:00'},
    {'tipo_operacao': 'expedição', 'tipo_produto': 'produto_acabado', 'quantidade': 3000, 'peso_total_kg': 1500, 'turno': 'manhã', 'dia_semana': 'segunda', 'hora_inicio': '09:00'},
    {'tipo_operacao': 'recebimento', 'tipo_produto': 'paletizado', 'quantidade': 800, 'peso_total_kg': 6000, 'turno': 'manhã', 'dia_semana': 'segunda', 'hora_inicio': '10:30'},
    # adicione mais linhas conforme o cenário
])

# Converter hora_inicio para minutos
operacoes_turno["hora_minutos"] = operacoes_turno["hora_inicio"].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))

# Remover coluna não usada
X_simulacao = operacoes_turno.drop("hora_inicio", axis=1)

# Prever tempo estimado de cada operação
tempos_previstos = modelo.predict(X_simulacao)
operacoes_turno["tempo_previsto_min"] = np.round(tempos_previstos, 2)
fator_escala = (operacoes_turno["quantidade"] / 1000).clip(lower=1)
operacoes_turno["tempo_previsto_corrigido"] = operacoes_turno["tempo_previsto_min"] * fator_escala

# 🔢 Somar tempo total do turno
tempo_total_turno = operacoes_turno["tempo_previsto_corrigido"].sum()

# ⚠️ Definir tempo de trabalho de um operador no turno (em minutos)
tempo_util_operador = 6.5 * 60  # 6h30min = 390 minutos úteis

# 🧮 Calcular necessidade de operadores
operadores_necessarios = np.ceil(tempo_total_turno / tempo_util_operador)

# 📊 Resultado
print("📦 Simulação de dimensionamento de equipe logística (turno da manhã):\n")
print(operacoes_turno[["tipo_operacao", "tipo_produto", "quantidade", "tempo_previsto_min"]])
print(f"\n⏱️ Tempo total previsto do turno: {tempo_total_turno:.1f} minutos")
print(f"👷 Operadores necessários para dar conta da demanda: {int(operadores_necessarios)} operador(es)")

📦 Simulação de dimensionamento de equipe logística (turno da manhã):

  tipo_operacao     tipo_produto  quantidade  tempo_previsto_min
0   recebimento    matéria-prima       15000              118.02
1     expedição  produto_acabado        3000              107.94
2   recebimento       paletizado         800               91.02

⏱️ Tempo total previsto do turno: 2185.1 minutos
👷 Operadores necessários para dar conta da demanda: 6 operador(es)


## ✅ Conclusão

Este notebook mostra como é possível aplicar inteligência artificial, mesmo com dados simples, para prever tempos operacionais, identificar gargalos e simular o dimensionamento ideal de equipes em armazéns.

Ferramentas como essa podem apoiar supervisores, analistas e gestores na tomada de decisão diária, com base em dados reais.